In [130]:
# Dependencies and Setup
import pandas as pd


In [131]:
# File to Load 
hero_path = "HeroesOfPymoli/Resources/purchase_data.csv"


In [132]:
hero_df=pd.read_csv(hero_path)
hero_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [133]:
hero_df.tail()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
775,775,Aethedru70,21,Female,60,Wolf,3.54
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46
778,778,Sisur91,7,Male,92,Final Critic,4.19
779,779,Ennrian78,24,Male,50,Dawn,4.60


In [134]:
hero_df.dtypes

Purchase ID      int64
SN              object
Age              int64
Gender          object
Item ID          int64
Item Name       object
Price          float64
dtype: object

In [135]:
hero_df.shape

(780, 7)

In [136]:
hero_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780 entries, 0 to 779
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Purchase ID  780 non-null    int64  
 1   SN           780 non-null    object 
 2   Age          780 non-null    int64  
 3   Gender       780 non-null    object 
 4   Item ID      780 non-null    int64  
 5   Item Name    780 non-null    object 
 6   Price        780 non-null    float64
dtypes: float64(1), int64(3), object(3)
memory usage: 42.8+ KB


In [141]:
# Player Count
# Display the total number of players
# I want to count the number of unique SN values and display that number as a new table dataframe with just the index 
# zero and column heading Total Players. Create a dictionary (key is Total Players: value is the count of the total number 
# of unique players), convert it to a DataFrame, and print the new DF. 
#total_players={"Total Players":[len(hero_df["SN"].value_counts())]}
#total_players_df=pd.DataFrame(total_players)
tp_df=pd.DataFrame({"Total Players":[len(hero_df["SN"].value_counts())]})
tp_df
#total_players_df

,Total Players
0,576


In [138]:
# Purchasing Analysis (Total):
# 1. Number of Unique Items

# 2. Average Purchase Price

# 3. Total Number of Purchases

# 4. Total Revenue


In [5]:
# Gender Demographics:
# 1. Percentage and Count of Male Players

# 2. Percentage and Count of Female Players

# 3. Percentage and Count of Other / Non-Disclosed


In [6]:
# Gender Demographics:
# 1. Percentage and Count of Male Players

# 2. Percentage and Count of Female Players

# 3. Percentage and Count of Other / Non-Disclosed


In [7]:
# Age Demographics:
# 1. The below each broken into bins of 4 years (i.e. <10, 10-14, 15-19, etc.)

# 2. Purchase Count

# 3. Average Purchase Price

# 4. Total Purchase Value

# 5. Average Purchase Total per Person by Age Group


In [8]:
# Top Spenders:
# 1. Identify the the top 5 spenders in the game by total purchase value, then list (in a table):

# 2. SN

# 3. Purchase Count

# 4. Average Purchase Price

# 5. Total Purchase Value


In [9]:
# Most Popular Items:
# 1. Identify the 5 most popular items by purchase count, then list (in a table):

# 2. Item ID

# 3. Item Name

# 4. Purchase Count

# 5. Item Price

# 6. Total Purchase Value


In [10]:
# Most Profitable Items:
# 1. Identify the 5 most profitable items by total purchase value, then list (in a table):

# 2. Item ID

# 3. Item Name

# 4. Purchase Count

# 5. Item Price

# 6. Total Purchase Value


In [ ]:
# As final considerations:
# You must use the Pandas Library and the Jupyter Notebook.

# You must submit a link to your Jupyter Notebook with the viewable Data Frames.

# You must include a written description of three observable trends based on the data.

# See Example Solution for a reference on expected format.
